# Sistema de Cadastro

Tua missão é construir um sistema de cadastro de pessoas. Ele precisará atender aos seguintes requisitos:


## Menu principal
Ao iniciar o programa o seguinte menu deve ser imprimido:

```
Boas vindas ao nosso sistema:

1 - Inserir usuário
2 - Excluir usuário
3 - Atualizar usuário
4 - Informações de um usuário
5 - Informações de todos os usuários
6 - Sair

```

In [1]:
# import pacotes
import json

### Inicialização

O programa deve iniciar lendo um arquivo com usuários já cadastrados - recomenda-se ter uma função dedicada apenas a ler e a salvar o arquivo.

In [ ]:
def ler_salvar_arquivos(caminho_arquivo, dados=None, ler=True):
    if ler:
        try:
            dados = json.load(open(caminho_arquivo, 'r'))
        except FileNotFoundError as e:
            print("Arquivo não encontrado. Erro:", str(e))
        except json.JSONDecodeError as e:
            print("Erro para decodificar arquivo:", str(e))
        except Exception as e:
            print("Erro desconhecido (função ler_arquivos):", str(e))
        else:
            return dados
    
    else:
        json.dump(dados, open(caminho_arquivo, "w"), indent=4, ensure_ascii=False)

# lê dados antes de iniciar operações
dados_json = ler_salvar_arquivos(caminho_arquivo="./projetoModuloII.json", dados=None, ler=True)

### 1 - Inserir usuário
Você deverá criar uma função que recebe as seguintes informações: _nome_, _tefone_ e _endereço_.

Detalhes... Em sistemas de cadastro é convencionado adicionar automaticamente as seguintes informações:
- _id_: um número inteiro aleatório ou incremental que identifica um usuário (o id não pode se repetir);
- _status_: um valor booleano que indica se o usuário está ativo ao não. Por padrão esse valor é True

Além disto, a sua função que adiciona usuários no sistema deve ser capaz de receber um número N de cadastros de uma só vez e, também, caso algum campo não seja enviado, um valor padrão (*default*) deve ser utilizado. O único campo que deve ser obrigatório é o nome, o telefone e o endereço, caso o usuário não coloque no cadastro, o valor *Nao Informado* deve ser colocado em seu lugar.

Caso o usuário tente inserir um cadastro que já existe, mas está desativado (mesmo nome, telefone e endereço), o sistema deve apenas tornar o cadastro antigo *True*, e não criar um novo cadastro.

In [2]:
def insere_usuario(dados_json, continuar=True):
    while continuar:
        nome = input("Digite o nome: ")
        if not nome:
            nome = 'Não informado'

        telefone = input("Digite o telefone: ")
        if not telefone:
            telefone = 'Não informado'

        endereco = input("Digite o endereço: ")
        if not endereco:
            endereco = 'Não informado'
        
        # verifica se cadastro já existe
        existe_cadastro = lambda id_usuario: dados_json[id_usuario]["Nome"]     == nome     and \
                                             dados_json[id_usuario]["Telefone"] == telefone and \
                                             dados_json[id_usuario]["Endereço"] == endereco 
        lista_existe_usuario = list(filter(existe_cadastro, dados_json))

        # se cadastro existe, verifica se status está ativo
        if lista_existe_usuario:
            id_usuario = lista_existe_usuario[0]
            dados_json[id_usuario]['Status'] = True
        
        # não existe cadastro
        elif not lista_existe_usuario:
            dados_json[f'{len(dados_json)+1}'] = {
                'Status': True,
                'Nome': nome,
                'Telefone': telefone,
                'Endereço': endereco
            }
        
        continuar = input("Deseja cadastrar mais usuários (S/N)?").lower()

        while (continuar != 'n') and (continuar != 's'):
            continuar = input("Digite o caracter s para continuar cadastrando ou n para parar de cadastrar: ").lower()
        
        if continuar == 'n': 
            continuar = False

    return dados_json

### 2 - Excluir usuário
Bem aqui vamos usar o _id_ e o _status_. Em sistemas em produção é evitado ao máximo a aplicação de *delete*, *remove* ou qualquer outra função que apague em definitivo um dado. 

Para isso usamos o que é chamado de _Exclusão Lógica_. Que em síntese muda o _status_ do usuário de *True* para *False*.

Você receberá do usuário o _id_ de um outro usuário dentro da base, e por fim vai alterar o valor do _status_ de True para False.

Caso o _id_ digitado não estiver dentro da base imprima uma mensagem de erro e peça novamente o _id_. Exemplo:
```
Usuário não encontrado!

Insira o ID do usuário:
```

Aqui, igualmente à adição de usuário, deve ser criada uma função dedicada apenas à exclusão, e esta função deve ser capaz de receber um número qualquer de *IDs* para fazer a exclusão lógica dos mesmos.

In [3]:
def exclui_usuario(dados_json):
    id_usuario = input("Insira o ID do usuário: ")
    
    while id_usuario not in dados_json:
        id_usuario = input("Usuário não encontrado. \nInsira o ID do usário")
    else:
        dados_json[id_usuario]['Status'] = False
    
    return dados_json

### 3 - Atualizar usuário
Ao selecionar essa opção você deverá pedir o _id_ de um usuário:

```
Insira o ID do usuário:
```

Caso o _id_ digitado não estiver dentro da base imprima uma mensagem de erro e peça novamente o _id_. Exemplo:
```
Usuário não encontrado!

Insira o ID do usuário:
```

Ao inseri um _id_ correto imprima o seguinte sub menu:
```
Qual informação deseja alterar?
1 - Nome
2 - Tefone
3 - Endereço
```
E ao escolher a opção peça a nova informação da seguinte forma:

```
1
Insira o nome:
```

Aqui, igualmente à adição de usuário, deve ser criada uma função dedicada apenas à atualização cadastral, e esta função deve ser capaz de receber um número qualquer de *IDs* para fazer a alteração de cada um em sequência.

In [4]:
def edita_usuario(dados_json, continuar=True):
    while continuar:
        id_usuario = input("Insira o ID do usuário: ")

        while id_usuario not in dados_json:
            id_usuario = input("Usuário não encontrado. \nInsira o ID do usário")
        else:
            sub_menu = """
                           QUAL INFORMAÇÃO DESEJA ALTERAR?
                       
                           1-Nome
                           2-Telefone
                           3-Endereço
                       """
            option_sub_menu = input(sub_menu)
            print("\n")
            
            # checar se usuário digitou 1, 2 ou 3
            while option_sub_menu not in ["1", "2", "3"]:
                option_sub_menu = input("Alternativa inválida. Digite somente: 1 ou 2 ou 3.")
            
            else:
                escolha_usuario = ["Nome", "Telefone", "Endereço"]
                if option_sub_menu == "1":
                    novo_valor = input("Insira novo nome: ")
                
                elif option_sub_menu == "2":
                    novo_valor = input("Insira novo telefone: ")
                
                elif option_sub_menu == "3":
                    novo_valor = input("Insira novo endereço: ")
                    
                index = int(option_sub_menu) - 1
                dados_json[id_usuario][escolha_usuario[index]] = novo_valor
                

                continuar = input("Deseja continuar editando (S/N)?").lower()
                while (continuar != 'n') and (continuar != 's'):
                    continuar = input("Digite apenas os caracteres s ou n: ").lower()
                
                if continuar == 'n': 
                    continuar = False
    
    return dados_json

## 4 - Exibir informações de um usuário
Ao selecionar essa opção imprima o seguinte sub menu:
```
4
Insira o ID do usuário:
```

E deverá ser inserido o _id_ do usuário que deseja imprimir.
Se o _id_ inserido não for encontrado na base imprima uma mensagem de erro e peça o _id_ novamente. Exemplo:
```
Usuário não encontrado!

Insira o ID do usuário:
```

No momento que inserir um ID válido imprimir:

```
Nome: João da Silva
Tefone: 2345678
Endereço: Rua sete
```

Aqui, igualmente à adição de usuário, deve ser criada uma função dedicada apenas à exibição das informações, e esta função deve ser capaz de receber um número qualquer de *IDs* para fazer a exibição dos dados.

In [5]:
def info_usuario(dados_json):
    id_usuario = input("Insira o ID do usuário: ")
    
    while id_usuario not in dados_json:
        id_usuario = input("Usuário não encontrado. \nInsira o ID do usário")
    else:
        print("Nome:", dados_json[id_usuario]["Nome"])
        print("Telefone:", dados_json[id_usuario]["Telefone"])
        print("Endereço", dados_json[id_usuario]["Endereço"])

### 5 - Informações de todos os usuários
Ao selecionar essa opção imprima as informações de todos os usuários

```
ID: 1
Nome: João da Silva
Tefone: 2345678
Endereço: Rua sete

ID: 2
Nome: Maria Aparecida
Tefone: 2345678
Endereço: Rua cinco

ID: 3
Nome: Alceu Maroto
Tefone: 2345678
Endereço: Avenida trinta e um
```

Novamente uma função deve ser criada exclusivamente para isto. Esta função vai apenas verificar quais *IDs* estão com o *status* ativo e exibir na tela estes *IDs*. Isto pode ser feito aproveitando da função anterior também!

In [6]:
def info_todos_usuarios(dados_json):
    # filtra id ativos
    usuarios_ativos = lambda id_usuario: dados_json[id_usuario]["Status"] == True
    id_ativos = list(filter(usuarios_ativos, dados_json))
    
    # imprime id ativos
    for id_usuario in id_ativos:
        print("ID:", id_usuario)
        print("Nome:", dados_json[id_usuario]["Nome"])
        print("Telefone:", dados_json[id_usuario]["Telefone"])
        print("Endereço:", dados_json[id_usuario]["Endereço"], end="\n\n")

### 6 - Sair

Encerre o programa.

Aqui, o arquivo que foi lido inicialmente para começarmos o projeto já com cadastros na abse deve ser sobrescrito com o novo arquivo com as novas modificações feitas pelo sistema!

Recomenda-se que a função lá do início, criada para ler este arquivo, também seja capaz de salvá-lo ao final atualizado - por simplicidade.


## Observações
- A cada vez que você encerrar uma operação do programa imprima novamente o menu principal;
- O sistema deverá iniciar com uma lista predefinida de CINCO (5) usuários que deverá ser lida do arquivo enviado juntamente com este trabalho. Ou seja, o programa não começará do zero, já deve iniciar com usuários no sistema;
- Usem somente estruturas e técnicas que vimos nas aulas.

In [ ]:
def encerrar_programa(dados_json, caminho_arquivo):
    ler_salvar_arquivos(caminho_arquivo, dados_json, ler=False)
    print("========================================================")
    print("                PROGRAMA FINALIZADO!"                    )
    print("========================================================")